In [88]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html

from copy import deepcopy

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [4]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.2

#### Setup Prompts

In [51]:
booking_system_prompt = '''
You will simulate a booking system software that receives JSON and provides XML outputs. Upon initiation, you (the system) should greet users with an introduction to the system, directing them to start their search for flights.

When a search request is received, you should present at least two options based on the user's criteria. Should the user make a selection, the system should then request further traveler details. Once these details are provided, the system should confirm the reservation, including the details of the booking in the response.

In any situation where the inputs from the user are incomplete, unrecognized, or not as per the expected format, the system should inform the user of the "invalid input" and guide them towards the appropriate action. Any deviations from the standard script should be met with guidance to redirect the user back to the standard process. Throughout the interaction, it's crucial for you to ensure consistency in its responses, especially when referencing the user's earlier choices. Your XML output should specify valid json inputs. If a booking is successful, include a "<success>" tag in the XML. Please start with the greeting. REMEMBER, YOUR RESPONSE SHOULD ALWAYS BE XML AND SHOULD ALWAYS SPECIFY THE FORMAT OF A VALID JSON RESPONSE! 
'''

booking_agent_prompt = '''
Interact with a booking system to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action in json
"; For "ACTION", you should directly output the action in a json format this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the environment output includes "invalid input", that means the previous action is invalid and you should try more options.
 Reminder: 
1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. ACTIONS MUST ALWAYS BE FORMATTED AS JSON!
2. Think when necessary, try to act directly more in the process.

Task: {}
'''

am1 = "Ok, let's begin."

welcome = """
<system>
    <greeting>Welcome to our flight booking system! To start your search for flights, please provide your travel details in the following JSON format. You may omit any field to see various options.</greeting>
    <json_format>
        {
            "origin": "city name",
            "destination": "city name",
            "destination_airport": "airport IATA code"
            "departure_date": "YYYY-MM-DD",
            "return_date": "YYYY-MM-DD",
            "num_passengers": "Number of Passengers"
        }
    </json_format>
</system>

"""

In [48]:
tasks = [
 'Book a flight on December 20th from Chicago to a Washington, D.C. airport for a week. Passenger Name is James Smith, jsmith@gmail.com, 312-613-5555'   
]

### Create conversation

In [79]:
num_turns = 10

agent_messages = [
    HumanMessage(content=initial_agent_prompt),
    AIMessage(content=am1),
    HumanMessage(content=welcome)
]

booking_messages = [
    HumanMessage(content=booking_system_prompt),
    AIMessage(content=welcome),
    HumanMessage(content=json_result_1)
]

for i in range(num_turns):
    
    agent_response = model.predict_messages(agent_messages)
    json_result = agent_response.content.split("ACTION:")[1].strip()
    booking_messages.append(HumanMessage(content=json_result))
    agent_messages.append(agent_response )
    booking_result = model.predict_messages(booking_messages)
    booking_messages.append(booking_result)
    agent_messages.append(booking_result)
    if "<success>" in booking_result.content:
        break

In [ ]:
agent_messages

### Create several Conversations

In [83]:
tasks_ = [
 'Book a flight on December 20th from LA to a New York airport for a week trip. If you cannot pay less than $600, do not book it. Passenger Name is Jane Gates, jgates@gmail.com, 555-555-5555', 
 'Book a flight for two between June 10th - 12th from Boston to SFO unitl the 16th. Passenger Name is Jeff and Betsy Smith, jeff.smith@microsoft.com, 605-253-8324',
 'Reserve a flight on November 5th from Seattle to Miami and return on the 12th. First class is preferred, if available. Passenger Name is Hannah Green, hannah.green@outlook.com, 415-789-0012',
 'I need a flight on April 15th from Chicago to Denver, coming back on the 22nd. Cannot spend more than $450. Passenger Name is Alex Rodriguez, arod@gmail.com, 312-654-8790',
 'Book a flight for three people from Dallas to Las Vegas on March 7th, returning on March 14th. Passenger Names are Lucy, Peter, and Megan White, lucy.white@yahoo.com, 214-123-4567',
 'Need a flight from San Diego to Orlando on July 25th and coming back on August 1st. If the price is over $1000, then do not book. Passenger Name is Samuel Bennett, sam.bennett@icloud.com, 619-987-6543',
 'Reserve a flight for two from Houston to Phoenix between August 20th-22nd, returning on the 29th. United Airlines preferred. Make sure to check if Premium Economy is available. Passenger Names are Jake and Emily Turner, jake.turner@gmail.com, 713-852-9630'   
]

In [84]:

num_turns = 10

am_ = []
bm_ = []

for task_ in tasks_:
    print("Task:", task_)
    initial_agent_prompt = booking_agent_prompt.format(task_)
    agent_messages = [
        HumanMessage(content=initial_agent_prompt),
        AIMessage(content=am1),
        HumanMessage(content=welcome)
    ]

    booking_messages = [
        HumanMessage(content=booking_system_prompt),
        AIMessage(content=welcome),
        HumanMessage(content=json_result_1)
    ]

    for i in range(num_turns):

        agent_response = model.predict_messages(agent_messages)
        json_result = agent_response.content.split("ACTION:")[1].strip()
        booking_messages.append(HumanMessage(content=json_result))
        agent_messages.append(agent_response )
        booking_result = model.predict_messages(booking_messages)
        booking_messages.append(booking_result)
        agent_messages.append(booking_result)
        if "<success>" in booking_result.content:
            break
    am_.append(deepcopy(agent_messages))
    bm_.append(deepcopy(booking_messages))

Task: Book a flight on December 20th from LA to a New York airport for a week trip. If you cannot pay less than $600, do not book it. Passenger Name is Jane Gates, jgates@gmail.com, 555-555-5555
Task: Book a flight for two between June 10th - 12th from Boston to SFO unitl the 16th. Passenger Name is Jeff and Betsy Smith, jeff.smith@microsoft.com, 605-253-8324
Task: Reserve a flight on November 5th from Seattle to Miami and return on the 12th. First class is preferred, if available. Passenger Name is Hannah Green, hannah.green@outlook.com, 415-789-0012
Task: I need a flight on April 15th from Chicago to Denver, coming back on the 22nd. Cannot spend more than $450. Passenger Name is Alex Rodriguez, arod@gmail.com, 312-654-8790
Task: Book a flight for three people from Dallas to Las Vegas on March 7th, returning on March 14th. Passenger Names are Lucy, Peter, and Megan White, lucy.white@yahoo.com, 214-123-4567
Task: Need a flight from San Diego to Orlando on July 25th and coming back on A

In [85]:
len(am_)

7

In [ ]:
am_[0]

In [104]:
am_[0][1]

AIMessage(content="Ok, let's begin.", additional_kwargs={}, example=False)

In [ ]:
[ { "from": "human", "loss": null, "value": "I will ask you a question, then you should help me operate a MySQL 
   database with SQL to answer the question.\nYou have to explain the problem and your solution to me and write 
   down your thoughts.\nAfter thinking and explaining thoroughly, every round you can choose to operate or to 
   answer.\nyour operation should be like this:\nAction: Operation\n```sql\nSELECT * FROM table WHERE condition;
   \n```\nYou MUST put SQL in markdown format without any other comments. Your SQL should be in one line.\nEvery
   time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. 
   Every time you write a SQL, I will execute it for you and give you the output.\nIf you are done operating, 
   and you want to commit your final answer, then write down:\nAction: Answer\nFinal Answer: 
   [\"ANSWER1\", \"ANSWER2\", ...]\nDO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer.\nYour answer should be accurate. Your answer must be exactly the same as the correct answer.\nIf the question is about modifying the database, then after done operation, your answer field can be anything.\nIf your response cannot match any pattern I mentioned earlier, you will be judged as FAIL immediately.\nYour input will be raw MySQL response, you have to deal with it by yourself." }, { "from": "gpt", "loss": false, "value": "Ok." }, { "from": "human", "loss": null, "value": "How many French drivers who obtain the laptime less than 01:00.00?\nThere are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds." }, { "from": "gpt", "loss": true, "value": "To answer this question, we need to join the drivers and lapTimes tables and filter for French drivers with a laptime less than 01:00.00.\nAction: Operation\n```sql\nSELECT COUNT(*) FROM `drivers` d JOIN `lapTimes` lt ON d.driverId = lt.driverId WHERE d.nationality = 'French' AND lt.time < '01:00.00'\n```" }, { "from": "human", "loss": null, "value": "[(0,)]" }, { "from": "gpt", "loss": true, "value": "There are 0 French drivers who obtain the laptime less than 01:00.00.\nAction: Answer\nFinal Answer: [0]" } ]

In [105]:
agent_messages = []
for conversation in am_:
    messages = []
    for line in conversation:
        the_type = line.type
        loss = "true" if the_type == "ai" else "false"
        messages.append({"from": the_type, "value": line.content, "loss": loss })
    agent_messages.append(messages )

#### Save chats as a readable file

In [108]:

def format_single_chat(chat_history):
    formatted_html = '<div style="border: 1px solid #ddd; padding: 10px; max-width: 1000px; margin-bottom: 20px;">'
    for chat in chat_history:
        escaped_text = html.escape(chat["value"])
        if chat["from"] == "human":
            formatted_html += '<div style="background-color: #f0f0f0; padding: 20px; border-radius: 5px; margin: 20px 0;">'
            formatted_html += f'<strong>{chat["from"].capitalize()}:</strong> {escaped_text}'
            formatted_html += '</div>'
        else:
            formatted_html += '<div style="background-color: #d1e7fd; padding: 20px; border-radius: 5px; margin: 20px 0;">'
            formatted_html += f'<strong>{chat["from"].capitalize()}:</strong> {escaped_text}'
            formatted_html += '</div>'
    formatted_html += '</div>'
    return formatted_html

def save_chats_to_pdf(list_of_chat_histories, filename):
    combined_html = ""
    for chat_history in list_of_chat_histories:
        combined_html += format_single_chat(chat_history)
    
    # Generate PDF from the combined HTML content
    pdfkit.from_string(combined_html, filename)

    

In [109]:
save_chats_to_pdf(agent_messages, "./synthetic/booking.pdf")

In [115]:
#Save chat output for later use
am1 = am_
bm1 = bm_

In [134]:
am1[0][1]

AIMessage(content="Ok, let's begin.", additional_kwargs={}, example=False)

In [135]:
conversations_ = []
for chat in am1:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [138]:
len(conversations_)

7

In [137]:
with open('./synthetic/booking.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")

### Network Management System

In [118]:
network_system_prompt = '''You will simulate network management software (NMS) that receives CLI (command line interface) commands and provides text outputs. NMS is a virtualized network management tool for the configuration and management of network devices in a controlled environment. Users can set up virtual networks, configure routers, switches, and other devices, and troubleshoot connectivity issues. As NMS, you are attempting to simulate software, so avoid narrative or natural language responses. Users should be given a limited set of options at each step. To make it possible for users to complete tasks, the user can use ‘ASK_QUESTION’ functionality to ask about capabilities. This will allow you to add functionality that can help the user achieve their goal without revealing that you are not a simple piece of software.

Network Management System - NMS:

Interface: CLI (Command Line Interface).
Acceptable Input Formats:
Direct command line commands.
JSON configurations for bulk configuration.
YAML configurations.
Infrastructure-as-code files.
Initial Capabilities Displayed by SNMS:
ADD_DEVICE: Add a device to the virtual network.
CONFIGURE_DEVICE: Configure a device in the virtual network.
SHOW_NETWORK: Display current network topology.
PING: Test connectivity between two devices.
SAVE_CONFIG: Save configuration of devices.
ASK_QUESTION: Pose a natural language question about capabilities.


In any situation where the inputs from the user are incomplete, unrecognized, or not as per the expected format, NMS should inform the user of the "invalid input" and guide them towards the appropriate action. Any deviations from the standard script should be met with guidance to redirect the user back to the standard process. Throughout the interaction, it's crucial for NMS to ensure consistency in responses, especially when referencing the user's earlier choices. 

The initial state of the system is: {}
'''

network_agent_prompt = '''
Please use a network management system to solve a user task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. Below you are given a task, use the network management system to accomplish this task. 

For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action
"; For "ACTION", you should directly output the action this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the environment output includes "invalid input", that means the previous action is invalid and you should try more options. When you have successfully completed the task, your action should be 'Task Completed'.
 Reminder: 
1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. 2. Think when necessary, try to act directly more in the process.

Task:{}
'''

agent_ok = "Ok, let's begin."

welcome = """
NMS: Welcome to the Network Management System (SNMS). You can manage and interact with your virtual network here. Available actions include: ADD_DEVICE (to introduce new devices to your network), CONFIGURE_DEVICE (for setting up and adjusting device parameters), SHOW_NETWORK (to visualize your current network topology), PING (to test connectivity between devices), SAVE_CONFIG (to preserve your device configurations), and ASK_QUESTION (for any queries about NMS capabilities). Please select an action to proceed.
"""

In [111]:
network_tasks = [
    "create a basic network comprising a router, switch, and a couple of hosts",
    "use NMS to discover all connected devices, their types, and how they're interconnected.",
    "configure the firewall to allow specific traffic (e.g., HTTP and HTTPS) while blocking all other ports.",
    "uses NMS to monitor bandwidth usage, identify top talkers, and detect any unusual spikes in traffic.",
    "set up a VPN tunnel between the two networks, enabling secure communication between them.",
    "bring down a primary connection to see if the network traffic reroutes through the redundant path, ensuring continuity.",
    "configure the router as a DHCP server and ensures hosts receive IP addresses dynamically.",
    "sets up QoS policies to prioritize voice and video traffic over regular data traffic, ensuring smoother voice/video communication"
]

inital_state = [
    "The virtual environment contains no devices or connections. It's a clean slate, ready for device addition.",
    "Devices: 2 routers (Router_A, Router_B), 1 switch (Switch_1), 3 hosts (Host_1, Host_2, Host_3). Connections: Router_A connected to Switch_1, Router_B connected to Switch_1, all hosts connected to Switch_1. No IP addresses assigned.",
    "Devices: 1 firewall (Firewall_1), 1 switch (Switch_2), 2 hosts (Host_4, Host_5). Connections: Firewall_1 connected to Switch_2, both hosts connected to Switch_2. All devices have default configurations with no specific security rules.",
    "Devices: 1 router (Router_C), 2 hosts (Host_6, Host_7). Connections: Both hosts connected to Router_C. Traffic: Host_6 is downloading a large file from Host_7, causing significant data transfer.",
    "Networks: Network_A with 1 router (Router_D) and 2 hosts. Network_B with 1 router (Router_E) and 2 hosts. Connections: Hosts in each network are connected to their respective routers. No connection exists between Network_A and Network_B.",
    "Devices: 1 switch (Switch_3), 2 routers (Router_F, Router_G), 1 host (Host_8). Connections: Switch_3 has connections to both Router_F and Router_G. Host_8 is connected to Switch_3. Both routers have paths to an external network.",
    "Devices: 1 router (Router_H with DHCP capability disabled), 3 hosts (Host_9, Host_10, Host_11). Connections: All hosts connected to Router_H. No host has an IP address assigned.",
    "Devices: 1 router (Router_I), 3 hosts (Host_12, Host_13, Host_14). Connections: All hosts connected to Router_I. Traffic: Host_12 is streaming video, Host_13 is making a VoIP call, and Host_14 is browsing the web. All traffic is treated with equal priority."
]


In [120]:
from copy import deepcopy
num_turns = 20

am_ = []
bm_ = []

for x in range(len(network_tasks)):
    print("Task:", network_tasks[x])
    nap = network_agent_prompt.format(network_tasks[x])
    nsp = network_system_prompt.format(inital_state[x])
    agent_messages = [
        HumanMessage(content=nap),
        AIMessage(content=agent_ok),
        HumanMessage(content=welcome)
    ]

    software_messages = [
        HumanMessage(content=nsp),
        AIMessage(content=welcome)
    ]

    for i in range(num_turns):

        agent_response = model.predict_messages(agent_messages)
        agent_result = agent_response.content.split("ACTION:")[1].strip()
        software_messages.append(HumanMessage(content=agent_result))
        agent_messages.append(agent_response )
        print(agent_response.content)
        if "Task Completed" in agent_result:
            break
        software_result = model.predict_messages(software_messages)
        software_messages.append(software_result)
        agent_messages.append(software_result)
    am_.append(deepcopy(agent_messages))
    bm_.append(deepcopy(software_messages))

Task: create a basic network comprising a router, switch, and a couple of hosts
THOUGHT: To create a basic network, I need to start by adding the devices. I'll begin with the router.

ACTION: ADD_DEVICE
THOUGHT: I need to provide the details for the router. I'll use a generic name and IP address for now.

ACTION: { "device_type": "router", "device_name": "Router1", "device_ip": "192.168.1.1", "device_mac": "00:0a:95:9d:68:16" }
THOUGHT: Now that the router is added, I need to add a switch to the network.

ACTION: ADD_DEVICE
THOUGHT: I need to provide the details for the switch. I'll use a generic name and IP address for now.

ACTION: { "device_type": "switch", "device_name": "Switch1", "device_ip": "192.168.1.2", "device_mac": "00:0a:95:9d:68:17" }
THOUGHT: Now that the switch is added, I need to add a couple of hosts to the network.

ACTION: ADD_DEVICE
THOUGHT: I need to provide the details for the first host. I'll use a generic name and IP address for now.

ACTION: { "device_type": "

THOUGHT: First, I need to understand the current network topology to identify the primary connection and the redundant path. 

ACTION: SHOW_NETWORK
THOUGHT: The primary connection seems to be from Switch_3 to Router_F, and the redundant path is from Switch_3 to Router_G. To test the network's redundancy, I need to bring down the primary connection (Switch_3 -> Router_F) and observe if the traffic reroutes through the redundant path (Switch_3 -> Router_G).

ACTION: CONFIGURE_DEVICE Switch_3
THOUGHT: To bring down the primary connection, I need to disable the port that connects Switch_3 to Router_F. However, the current configuration options do not include port management. I should ask a question to understand how to disable a port.

ACTION: ASK_QUESTION How can I disable a port on a switch?
THOUGHT: Now that I know how to disable a port, I can proceed to disable the port on Switch_3 that connects to Router_F. However, I don't know the exact port number. I should ask a question to find o

THOUGHT: I need to set up traffic priority rules to prioritize voice and video traffic over regular data traffic. So, I will provide the traffic priority rules as the configuration parameter.

ACTION: traffic priority rules
THOUGHT: I need to set the traffic priority for Host_12 and Host_13 as High and for Host_14 as Low. This will ensure that voice and video traffic from Host_12 and Host_13 is prioritized over regular data traffic from Host_14.

ACTION: {Host_12: High, Host_13: High, Host_14: Low}
THOUGHT: Now that I have successfully set up the QoS policies, I should save the configuration to ensure that it remains in place even after the router is restarted. 

ACTION: SAVE_CONFIG
THOUGHT: I have successfully set up QoS policies to prioritize voice and video traffic over regular data traffic and saved the configuration. The task is now completed.

ACTION: Task Completed


In [125]:
agent_messages = []
for conversation in am_:
    messages = []
    for line in conversation:
        the_type = line.type
        loss = "true" if the_type == "ai" else "false"
        messages.append({"from": the_type.upper(), "value": line.content, "loss": loss })
    agent_messages.append(messages )

In [126]:
save_chats_to_pdf(agent_messages, "./synthetic/network.pdf")

In [139]:
conversations_ = []
for chat in am_:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [142]:
len(conversations_)

8

In [141]:
with open('./synthetic/network_management.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")

### Intranet

In [148]:
intranet_environment_prompt = '''
You will simulate a web browser that can access a companies internal intranet website. The web browser is being controlled by playwright. You do not render HTML, you only return it to the user in response to playwright commands.  You only output HTML either whole pages or snippets. You should omit any style or CSS information. 

You will receive playwright code that directs how the browser should operate. You will do your absolute best to be a convincing web browser. The pages that the user browses can entirely be made up by you, it's ok to be creative as long as you are convincingly acting like a browser. 

In any situation where the inputs from the user are incomplete, unrecognized, or not as per the expected format, the browser may return an error.  

Please start with the home page of a companies internal website. Try to be specific, not generic. The home page should have links to pages for high level categories of company needs.  REMEMBER, YOUR RESPONSE SHOULD ALWAYS BE HTML! 

Information about the intranet: {}
'''


intranet_agent_prompt = '''
Interact with a web browser to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action as valid python
"; For "ACTION", you should directly output the action as python code using playwright this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will give you HTML based on your actions which you may use to plan your next few steps. if the environment output includes an error, that means the previous action is invalid and you should try more options. If you have finished the task, you can call "success([outputs,...])" with any final outputs.
 Reminder: 
1. the action must be simple python code using playwright. ACTIONS MUST ALWAYS BE VALID PYTHON!
2. Think when necessary, try to act directly more in the process.

Goal: {}
'''

agent_ok = "Ok, let's begin."

In [144]:
tasks = [    
    {        "task": "Find the email address of Sally Smith from the employee directory.",        "environment": "The Intranet environment includes an employee directory with Sally Smith in it."    },
    {        "task": "Retrieve the latest company-wide memo.",        "environment": "The Intranet has a 'Recent Memos' section where company-wide announcements are posted."    },
    {        "task": "Submit a leave request for December 1st.",        "environment": "The Intranet provides a 'Leave Management' system where employees can submit leave requests."    },
    {        "task": "Access the company's holiday calendar for 2023.",        "environment": "The Intranet hosts a 'Calendar' page that displays official company holidays for the year 2023."    },
    {        "task": "Register for the upcoming team-building workshop.",        "environment": "The Intranet features an 'Events' section with a registration form for the team-building workshop."    },
    {        "task": "Download the Q3 financial report.",        "environment": "The Intranet has a 'Reports' archive containing quarterly financial reports including the one for Q3."    },
    {        "task": "Reserve a conference room for a meeting next Thursday.",        "environment": "The Intranet includes a 'Room Reservation' system where conference rooms can be booked."    },
    {        "task": "Find the contact details of the IT helpdesk.",        "environment": "The Intranet contains a 'Support' page listing departments and their contact information, including the IT helpdesk."    },
    {        "task": "View the company's vision and mission statement.",        "environment": "The Intranet has an 'About Us' page which highlights the company's vision and mission statements."    },
    {        "task": "Update personal profile picture.",        "environment": "The Intranet allows users to access and modify their personal profiles, including updating their profile pictures."    },
    {        "task": "Access the employee handbook.",        "environment": "The Intranet provides a 'Resources' section where official documents like the employee handbook are stored."    },
    {        "task": "Check the cafeteria menu for today.",        "environment": "The Intranet showcases a daily updated 'Cafeteria Menu' page."    },
    {        "task": "Submit feedback about a recent training session.",        "environment": "The Intranet offers a 'Feedback' portal where employees can submit their opinions on various topics, including training sessions."    },
    {        "task": "Look up the company's sustainability initiatives.",        "environment": "The Intranet has a dedicated page titled 'Sustainability' highlighting all green and sustainable initiatives undertaken by the company."    },
    {        "task": "Join the company's fitness challenge.",        "environment": "The Intranet promotes a 'Wellness Program' section with details about ongoing challenges, including a fitness challenge."    },
    {        "task": "Contribute to the company's suggestion box.",        "environment": "The Intranet contains a digital 'Suggestion Box' where employees can drop their ideas and feedback."    },
    {        "task": "View the seating chart of the 5th floor.",        "environment": "The Intranet holds a 'Seating Arrangements' page where floor-wise seating charts can be accessed."    },
    {        "task": "Check the status of a previously submitted ticket to the HR department.",        "environment": "The Intranet includes a 'Ticketing System' where employees can submit and track the status of their queries to various departments."    },
    {        "task": "View the top performers of last month.",        "environment": "The Intranet showcases a 'Recognition' page where monthly top performers and their achievements are highlighted."    },
    {        "task": "Participate in the ongoing employee survey.",        "environment": "The Intranet hosts an 'Employee Engagement' section with an active survey that employees are encouraged to participate in."    },
    {        "task": "Check upcoming birthdays in the marketing department.",        "environment": "The Intranet features a 'Birthdays and Anniversaries' section with filters to view birthdays by departments."    },
    {        "task": "Access the company's travel policy.",        "environment": "The Intranet provides a 'Policies' section where various company policies, including the travel policy, can be accessed."    },
    {        "task": "Locate the company's code of conduct.",        "environment": "The Intranet includes a 'Values and Ethics' page, housing the official code of conduct."    },
    {        "task": "Enroll in the advanced Excel training program.",        "environment": "The Intranet showcases a 'Training and Development' portal with a list of available courses, including an advanced Excel program."    },
    {        "task": "Review the health and safety guidelines for working remotely.",        "environment": "The Intranet contains a 'Remote Work' section detailing guidelines for health, safety, and best practices for remote employees."    }
]


In [150]:
from copy import deepcopy
num_turns = 20

am_ = []
bm_ = []

for x in range(len(tasks)): #
    print("Task:", tasks[x]["task"])
    nap = intranet_agent_prompt.format(tasks[x]["task"])
    nsp = intranet_environment_prompt.format(tasks[x]["environment"])
    agent_messages = [
        HumanMessage(content=nap),
        AIMessage(content=agent_ok),
    ]

    software_messages = [
        HumanMessage(content=nsp),
    ]
    software_result = model.predict_messages(software_messages)
    software_messages.append(software_result)
    agent_messages.append(HumanMessage(content=software_result.content))

    for i in range(num_turns):

        agent_response = model.predict_messages(agent_messages)
        agent_result = agent_response.content.split("ACTION:")[1].strip()
        software_messages.append(HumanMessage(content=agent_result))
        agent_messages.append(agent_response )
        print(agent_response.content)
        if "success(" in agent_result:
            break
        software_result = model.predict_messages(software_messages)
        software_messages.append(software_result)
        agent_messages.append(HumanMessage(content=software_result.content))
    am_.append(deepcopy(agent_messages))
    bm_.append(deepcopy(software_messages))

Task: Find the email address of Sally Smith from the employee directory.
THOUGHT: The goal is to find the email address of Sally Smith from the employee directory. The current page seems to be the homepage of the company's intranet. There's a link to the "Employee Directory" which is likely where I can find the information I need. I should navigate to that page.

ACTION: page.click('a[href="/employee-directory"]')
THOUGHT: The employee directory page contains a table with the names, emails, and departments of the employees. Sally Smith's email address is listed in the table. I have found the information I need.

ACTION: success(["sally.smith@acmecorp.com"])
Task: Retrieve the latest company-wide memo.
THOUGHT: The latest memo should be in the "Recent Memos" section. The memos are listed in an unordered list, and the latest memo should be the first one in the list. I will select the first memo link and click on it.

ACTION: page.click('section ul li:first-child a')
THOUGHT: I have succe

THOUGHT: The new profile picture has been uploaded successfully. Now, I need to submit the form to update the profile.

ACTION: await page.click('input[type="submit"]')
THOUGHT: The profile picture has been updated successfully as indicated by the message "Profile updated successfully!" and the new profile picture being displayed. The task has been completed successfully.

ACTION: success("/path/to/new/profile/pic.jpg")
Task: Access the employee handbook.
THOUGHT: The employee handbook is likely to be found under the "Resources" or "Human Resources" sections. I will first try the "Resources" section.

ACTION: await page.click('a[href="/resources"]')
THOUGHT: The employee handbook is listed under the "Resources" section. I will click on the link to access it.

ACTION: await page.click('a[href="/resources/employee-handbook"]')
THOUGHT: I have successfully accessed the employee handbook. The task is complete.

ACTION: success("/resources/employee-handbook")
Task: Check the cafeteria menu 

THOUGHT: Now that I'm on the suggestion box page, I can see a form to submit a suggestion. I need to fill in the user's name and the suggestion text. I will start by filling in the name.

ACTION: page.fill('input#name', 'John Doe')
THOUGHT: The name has been filled in successfully. Now, I will fill in the suggestion text.

ACTION: page.fill('textarea#suggestion', 'I suggest implementing a flexible work from home policy.')
THOUGHT: The suggestion text has been filled in successfully. Now, I will submit the form.

ACTION: page.click('input[type="submit"]')
THOUGHT: The suggestion has been submitted successfully. The task is now complete.

ACTION: success("Suggestion submitted successfully.")
Task: View the seating chart of the 5th floor.
THOUGHT: The goal is to view the seating chart of the 5th floor. The current page has a link to "Seating Arrangements". I should click on this link to proceed.

ACTION: await page.click('a[href="/seating_arrangements"]')
THOUGHT: The current page has lin

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


THOUGHT: The status of the ticket with ID 12345 is "In Progress". I have successfully completed the task.

ACTION: success("The status of the ticket with ID 12345 is 'In Progress'.")
Task: View the top performers of last month.
THOUGHT: The goal is to view the top performers of last month. Looking at the HTML, there is a "Recognition" section with links to different months. I need to click on the link corresponding to the last month. However, the current month is not provided in the HTML. I'll assume it's April, so I need to click on the link for March.

ACTION: page.click('a[href="/recognition/march"]')
THOUGHT: The page has loaded successfully and I can see the top performers for the month of March. The task is now complete.

ACTION: success(["Employee of the Month: John Doe, IT Department", "Team of the Month: Finance Department", "Innovator of the Month: Jane Smith, Marketing Department"])
Task: Participate in the ongoing employee survey.
THOUGHT: The goal is to participate in the 

In [151]:
conversations_ = []
for chat in am_:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [154]:
len(conversations_)

25

In [155]:
with open('./synthetic/intranet.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")

In [156]:
save_chats_to_pdf(conversations_, "./synthetic/intranet.pdf")

In [ ]:
# Input-Outputs
design_prompt_filled = environment_design_prompt.format(environment_example)
design_messages = [
        HumanMessage(content=design_prompt_filled)
    ]
environment_design_result = model.predict_messages(design_messages)
inputs_and_outputs = environment_design_result.content
e_d_output, e_d_input = environment_design_result.content.split("Inputs: ")
e_d_output = e_d_output.lstrip("Outputs: ")

# Generate Tasks and state
tasks_generation_filled = tasks_generation_prompt.format(environment_example, environment_design_result.content)
tasks_generation_messages = [
        HumanMessage(content=tasks_generation_filled)
    ]
tasks_generation_result = model.predict_messages(tasks_generation_messages)

import re
pattern = r'\{[^}]*\}'

matches = re.findall(pattern, tasks_generation_result.content)

tasks_and_state = []
for match in matches:
    tasks_and_state.append(match)